In [ ]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.neighbors import KernelDensity
from scipy import stats
from scipy.interpolate import interp1d

In [ ]:
data_path = Path("/bgfs/jnewman/bid13/photoZ/data/pasquet2019")

file_path = data_path/ "sdss_vagc.npz"

In [ ]:
data = np.load(file_path, allow_pickle=True)
cat = pd.DataFrame(data["labels"])

In [ ]:
plt.plot(cat.z, cat.dered_petro_r,".")

In [ ]:
mask_psqt = (cat["dered_petro_r"]<=17.8)&(cat['z']<=0.4)&( cat['primtarget'] >=64)

In [ ]:
plt.plot(cat.z[mask_psqt], cat.dered_petro_r[mask_psqt],".")

# Make data for MorphCaps-iter 0

In [ ]:
mask_gz = np.isfinite((cat[mask_psqt]).gz1Morpho)

In [ ]:
print(len(cat["z"][mask_psqt][mask_gz]))
plt.hist(cat["z"][mask_psqt][mask_gz],bins=50)
plt.yscale("log")
plt.show()

In [ ]:
np.savez(
    data_path / "sdss_gz1_final_iter0.npz",
    
    cube=(data["cube"][mask_psqt][mask_gz]).astype("float16"),
    labels=data["labels"]["gz1Morpho"][mask_psqt][mask_gz],
    specObjID=data["labels"]["specObjID"][mask_psqt][mask_gz],
    z=data["labels"]["z"][mask_psqt][mask_gz],
    cat=data["labels"][mask_psqt][mask_gz],
    
    cube_x=(data["cube"][mask_psqt][~mask_gz]).astype("float16"),
    specObjID_x=data["labels"]["specObjID"][mask_psqt][~mask_gz],
    z_x=data["labels"]["z"][mask_psqt][~mask_gz],
    cat_x=data["labels"][mask_psqt][~mask_gz],
)

### Add extra data after iter 0

In [ ]:
old_data = np.load(data_path/"sdss_gz1_raw_iter0.npz",allow_pickle=True)

In [ ]:
new_data = np.load("good_ids_raw_iter0.npz")
args = new_data["catID"]
new_labels = new_data["labels"]
new_specObjID = new_data["specObjID"]
non_args = new_data["badID"]

In [ ]:
np.savez(
    data_path / "sdss_gz1_iter1.npz",
    cube=np.concatenate([old_data["cube"], old_data["cube_x"][args]]),
    labels=np.concatenate([old_data["labels"], new_labels]),
    specObjID=np.concatenate([old_data["specObjID"], new_specObjID]),
    z=np.concatenate([old_data["z"], old_data["z_x"][args]]),
    cat=np.concatenate([old_data["cat"], old_data["cat_x"][args]]),
    
    cube_x=old_data["cube_x"][non_args],
    specObjID_x=old_data["specObjID_x"][non_args],
    z_x=old_data["z_x"][non_args],
    cat_x=old_data["cat_x"][non_args]
        )

# Scale input data

In [ ]:
data = np.load("/data/bid13/photoZ/data/pasquet2019/sdss_gz1_final_iter2.npz",allow_pickle=True)

In [ ]:
stat = np.load("img_stat.npz")
m = stat["mean"]
s = stat["std"]

In [ ]:
np.savez(
    "/data/bid13/photoZ/data/pasquet2019/sdss_gz1_final_iter2_mean_scaled.npz",
    cube=data["cube"].astype("float16")-m,
    labels=data["labels"],
    specObjID=data["specObjID"],
    z=data["z"],
    cat=data["cat"],
        )

In [ ]:
np.savez(
    "/data/bid13/photoZ/data/pasquet2019/sdss_gz1_final_iter2_mean_std_scaled.npz",
    cube=(data["cube"].astype("float16")-m)/s,
    labels=data["labels"],
    specObjID=data["specObjID"],
    z=data["z"],
    cat=data["cat"],
        )